In [1]:
import pandas as pd
import numpy as np
from talib import abstract

pd.set_option('display.max_rows', 10000)

In [2]:
from datetime import datetime
import time

def convert_date(exchange):
    exchange = exchange.rename(columns={'DateTime': 'Time'})
    exchange["Time"] = pd.to_datetime(exchange["Time"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    return exchange

def indicators(pair):
    initial = time.perf_counter()
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    print(currency.shape)
    print(currency['DateTime'].iloc[-1])
    start, end = find_interval_index(currency)
    print(start, end)
    currency = convert_date(currency)
    
    print(currency)
    currency = configure_time(15, currency)
    print(currency)
    
    currency['EMA_10'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=960))
    currency['EMA_50'] = pd.DataFrame(abstract.EMA(currency['Close'], timeperiod=4800))
    currency['RSI'] = pd.DataFrame(abstract.RSI(currency['Close'], timeperiod=14))
    currency['A/D Index'] = pd.DataFrame(abstract.AD(currency['High'], currency['Low'], currency['Close'], currency['Volume']))
    currency['A/D Index'] = currency['A/D Index'] - currency['A/D Index'].shift(1)
    currency = stationary_log_returns(currency)
    
    currency["Time"] = currency["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    print(currency)

    currency.to_csv("../data/processed/exchange_rate/{}_exchange.csv".format(pair), index=False)
    
    time_elapsed = time.perf_counter() - initial
    print(time_elapsed)
    
    return currency

def find_interval_index(currency):
    start = 0
    end = 0
    index = 0
    for date in currency["DateTime"]:
        if start == 0 and date[:4] == "2018":
            start = index
        elif end == 0 and date[:4] == "2021":
            end = index
            break;
        index += 1
    return start, end
    
def stationary_log_returns(pair):
    pair = pair.copy()
    pair['Real Close'] = pair['Close']
    pair["Close"] = np.log(pair["Close"]/pair["Close"].shift(1))
    pair["Open"] = np.log(pair["Open"]/pair["Open"].shift(1))
    pair["High"] = np.log(pair["High"]/pair["High"].shift(1))
    pair["Low"] = np.log(pair["Low"]/pair["Low"].shift(1))
    pair["EMA_10"] = np.log(pair["EMA_10"]/pair["EMA_10"].shift(1))
    pair["EMA_50"] = np.log(pair["EMA_50"]/pair["EMA_50"].shift(1))
    return pair

def configure_time(minutes, dataframe):
    time_frame = pd.date_range(start="2017-01-01 22:00:00", freq="{}T".format(minutes), end="2020-12-31 21:59:00")
    time_frame = pd.DataFrame(time_frame, columns=["Time"])
    time_frame["Time"] = time_frame["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    time_frame['Time'] = pd.to_datetime(time_frame['Time'], utc=True)
    dataframe['Time'] = dataframe['Time'].astype('datetime64[ns, UTC]')
    
    configured_df = time_frame.merge(dataframe, how="inner", on="Time")
    
    return configured_df

In [3]:
cool = indicators("AUDCAD")
print(cool)

(1556639, 6)
20210304 23:59:00.000
373054 1493308
                        Time  Volume     High      Low    Close     Open
0        2017-01-01 22:05:00       4  0.96863  0.96863  0.96863  0.96863
1        2017-01-01 22:07:00       4  0.96863  0.96858  0.96863  0.96860
2        2017-01-01 22:10:00       4  0.96888  0.96888  0.96888  0.96888
3        2017-01-01 22:14:00       4  0.96840  0.96840  0.96840  0.96840
4        2017-01-01 22:17:00       4  0.96837  0.96837  0.96837  0.96837
...                      ...     ...      ...      ...      ...      ...
1556634  2021-03-04 23:55:00     104  0.97773  0.97758  0.97760  0.97771
1556635  2021-03-04 23:56:00     120  0.97777  0.97759  0.97769  0.97761
1556636  2021-03-04 23:57:00     104  0.97805  0.97767  0.97799  0.97767
1556637  2021-03-04 23:58:00      64  0.97810  0.97792  0.97802  0.97798
1556638  2021-03-04 23:59:00      48  0.97808  0.97800  0.97804  0.97804

[1556639 rows x 6 columns]
                           Time  Volume     Hi

In [4]:
cool = cool.iloc[40000:40100]

In [5]:
cool

,Time,Volume,High,Low,Close,Open,EMA_10,EMA_50,RSI,A/D Index,Real Close
40000,2018-08-09 17:00:00,112,-0.000321,-0.000342,-0.000187,-0.000415,-0.000002,-0.000003,36.688064,7.840000e+01,0.96468
40001,2018-08-09 17:15:00,64,-0.000518,-0.000436,-0.000487,-0.000415,-0.000003,-0.000004,31.325045,6.400000e+01,0.96421
40002,2018-08-09 17:30:00,60,0.000239,0.000259,0.000145,0.000301,-0.000003,-0.000003,34.401113,-4.800000e+01,0.96435
40003,2018-08-09 17:45:00,60,-0.000373,-0.000353,-0.000322,-0.000404,-0.000004,-0.000004,31.081290,0.000000e+00,0.96404
40004,2018-08-09 18:00:00,108,0.000135,0.000052,0.000104,0.000166,-0.000004,-0.000004,33.316826,1.350000e+01,0.96414
40005,2018-08-09 18:15:00,84,-0.000342,-0.000332,-0.000384,-0.000363,-0.000004,-0.000004,29.503496,-3.920000e+01,0.96377
40006,2018-08-09 18:30:00,92,-0.000425,-0.000374,-0.000415,-0.000477,-0.000005,-0.000004,26.034294,-9.200000e+01,0.96337
40007,2018-08-09 18:45:00,68,-0.000135,-0.000218,-0.000197,-0.000062,-0.000006,-0.000004,24.557184,-5.288889e+01,0.96318
40008,2018-08-09 19:00:00,120,-0.000332,-0.000478,-0.000239,-0.000654,-0.000006,-0.000004,22.865908,6.750000e+01,0.96295
40009,2018-08-09 19:15:00,96,0.000021,0.000042,-0.000145,0.000218,-0.000006,-0.000004,21.878192,-5.120000e+01,0.96281


In [6]:
cool['Real Close'].mul(np.exp(cool['Close'].shift(-1))).shift(1)

40000        NaN
40001    0.96421
40002    0.96435
40003    0.96404
40004    0.96414
40005    0.96377
40006    0.96337
40007    0.96318
40008    0.96295
40009    0.96281
40010    0.96288
40011    0.96243
40012    0.96217
40013    0.96227
40014    0.96191
40015    0.96218
40016    0.96196
40017    0.96181
40018    0.96203
40019    0.96227
40020    0.96230
40021    0.96233
40022    0.96234
40023    0.96188
40024    0.96166
40025    0.96145
40026    0.96134
40027    0.96148
40028    0.96164
40029    0.96163
40030    0.96151
40031    0.96166
40032    0.96176
40033    0.96142
40034    0.96166
40035    0.96135
40036    0.96182
40037    0.96189
40038    0.96197
40039    0.96146
40040    0.96146
40041    0.96149
40042    0.96174
40043    0.96147
40044    0.96147
40045    0.96111
40046    0.96116
40047    0.96108
40048    0.96008
40049    0.95885
40050    0.95823
40051    0.95887
40052    0.95879
40053    0.95776
40054    0.95644
40055    0.95758
40056    0.95718
40057    0.95640
40058    0.956